## Adım 1: Gerekli Araçlar ve Veri Yükleme

In [5]:
# Gerekli Kütüphaneler
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Veri Yükleme ve Temizlik
try:
    df = pd.read_csv('../data/US_youtube_trending_data.csv', encoding='utf-8')
except Exception:
    df = pd.read_csv('../data/US_youtube_trending_data.csv', encoding='latin1')

# Log dönüşümü için 0 olan değerleri temizliyoruz (np.log1p'nin düzgün çalışması için)
df = df[(df['view_count'] > 0) & (df['likes'] > 0)]

### Veri İşleme Kararı (Preprocessing Rationale)

**1. Logaritmik Dönüşüm:**
İzlenme sayıları (`view_count`) **aşırı çarpık (skewed)** ve **uç değerler** (viral videolar) içerdiği için, modelin bu uç değerlere sapmasını engellemek amacıyla **Logaritmik Dönüşüm** (`np.log1p`) uygulanmıştır. Bu dönüşüm, veriyi Doğrusal Regresyon (`Linear Regression`) için uygun olan **Normal Dağılıma** yaklaştırır.

**2. Sıfır Değer Temizliği:**
Trend olma kriterini karşılamayan veya hatalı çekilmiş veriyi temizlemek amacıyla, izlenme veya beğeni sayısı sıfır olan tüm satırlar (**Log dönüşümünden önce**) veri setinden çıkarılmıştır. Bu, modelin eğitildiği verinin kalitesini artırmak için zorunlu bir temizlik adımıdır.

## Adım 2 : Özellik Seçimi ve Veri Bölme (Feature Selection & Split)

In [6]:
# Baseline için sadece en güçlü korelasyona sahip ham sayısal özellikleri kullanıyoruz.
features = ['likes', 'comment_count']
target = 'view_count'

# Log dönüşümü uygulanmış hedef değişken
X = df[features]
y = np.log1p(df[target]) 

# Veriyi Eğitim (%80) ve Test (%20) olarak ayırıyoruz.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Eğitim Seti Boyutu: {X_train.shape}")
print(f"Test Seti Boyutu: {X_test.shape}")

Eğitim Seti Boyutu: (213780, 2)
Test Seti Boyutu: (53446, 2)


## Adım 3: Algoritma Seçimi (Linear Regression)
Baseline modeli için, regresyon problemlerinde en basit ve en hızlı çalışan algoritma olan **Doğrusal Regresyon (Linear Regression)** tercih edilmiştir. Bu algoritma, karmaşık özellikler (Başlık, Gün, Saat) eklenmeden elde edilebilecek en temel başarı skorunu temsil edecektir.

###  Metrik Seçimi ve Model Eğitimi Rasyoneli

**1. R2 Skoru Kullanım Gerekçesi:**
Regresyon modelimizin başarısını ölçmek için ana metrik olarak **R2 Skoru (Belirlilik Katsayısı)** tercih edilmiştir. R2, kurulan modelin hedef değişkendeki (`view_count`) toplam değişimi ne kadar iyi açıkladığını 0 ile 1 arasında bir değerle gösterir. Bu metrik, ilerideki Feature Engineering adımlarında elde edeceğimiz daha karmaşık modellerin (XGBoost, Random Forest) bu temel referans skoru ne kadar geçtiğini yüzde bazında kanıtlamak için en uygun araçtır.

**2. Model Eğitimi:**
Baseline modeli, Log dönüşümünden geçmiş `likes` ve `comment_count` gibi ham özellikler kullanılarak **Doğrusal Regresyon (Linear Regression)** algoritmasıyla eğitilmiştir.

In [7]:
# Model Eğitimi
model = LinearRegression()
model.fit(X_train, y_train) 

# Tahmin Yap ve Skorları Hesapla
y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"\n--- BASELINE MODEL SKORLARI ---")
print(f"R2 Skoru (Başarı): {r2:.4f}")
print(f"RMSE (Hata): {rmse:.4f}")

# Örnek Karşılaştırma
comparison = pd.DataFrame({
    'Gerçek İzlenme': np.expm1(y_test), 
    'Model Tahmini': np.expm1(y_pred)
})
print("\nÖrnek Tahminler:")
display(comparison.head(5))


--- BASELINE MODEL SKORLARI ---
R2 Skoru (Başarı): 0.2897
RMSE (Hata): 0.9632

Örnek Tahminler:


,Gerçek İzlenme,Model Tahmini
268006,1143117.0,9.537387e+05
229778,25215180.0,3.023812e+06
59867,1073464.0,9.762569e+05
143838,146108.0,9.017340e+05
57776,1435404.0,9.947898e+05
